In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("mytree").getOrCreate()

21/11/12 11:14:05 WARN Utils: Your hostname, xtian-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.39 instead (on interface enp5s0)
21/11/12 11:14:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/12 11:14:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.ml import pipeline

In [24]:
from pyspark.ml.classification import (RandomForestClassifier,
                            GBTClassifier,
                        DecisionTreeClassifier)


In [6]:
#from pyspark.ml.regression import DecisionTreeRegressor

In [7]:
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

21/11/12 11:17:35 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [28]:
train_data, test_data = data.randomSplit([0.7, .03])

In [20]:
dtc = DecisionTreeClassifier(featuresCol='features', 
                             labelCol='label')

In [21]:
rfc = RandomForestClassifier(featuresCol='features', 
                             labelCol='label',
                            numTrees=100)

In [26]:
gbtc = GBTClassifier(featuresCol='features', 
                             labelCol='label')

In [40]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbtc_model = gbtc.fit(train_data)

In [41]:
dtc_pred = dtc_model.transform(test_data)
rfc_pred = rfc_model.transform(test_data)
gbtc_pred = gbtc_model.transform(test_data)

In [42]:
dtc_pred.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[154,155,156...|   [42.0,0.0]|  [1.0,0.0]|       0.0|
|  1.0|(692,[123,124,125...|   [0.0,54.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[129,130,131...|   [42.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+



In [43]:
rfc_pred.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[154,155,156...|   [96.0,4.0]|[0.96,0.04]|       0.0|
|  1.0|(692,[123,124,125...|   [8.0,92.0]|[0.08,0.92]|       1.0|
|  1.0|(692,[129,130,131...|  [48.0,52.0]|[0.48,0.52]|       1.0|
+-----+--------------------+-------------+-----------+----------+



In [44]:
gbtc_pred.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[154,155,156...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  1.0|(692,[123,124,125...|[-1.5435020027249...|[0.04364652142729...|       1.0|
|  1.0|(692,[129,130,131...|[1.22256351403293...|[0.92020436871057...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+



In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [46]:
acc_eval = MulticlassClassificationEvaluator(metricName="accuracy")

In [50]:
print("DTC ACC")
acc_eval.evaluate(dtc_pred)

DTC ACC


0.6666666666666666

In [51]:
print("RFC ACC")
acc_eval.evaluate(rfc_pred)

RFC ACC


1.0

In [52]:
print("GBTC ACC")
acc_eval.evaluate(gbtc_pred)

GBTC ACC


0.6666666666666666

In [53]:
rfc_model.featureImportances

SparseVector(692, {119: 0.0012, 147: 0.0007, 153: 0.0004, 208: 0.001, 209: 0.0004, 212: 0.001, 214: 0.0009, 216: 0.0122, 217: 0.0019, 234: 0.0004, 242: 0.0014, 243: 0.0005, 244: 0.0166, 245: 0.0005, 258: 0.0004, 259: 0.0003, 262: 0.0002, 263: 0.0153, 266: 0.0005, 267: 0.0029, 268: 0.0024, 272: 0.0079, 273: 0.0098, 274: 0.0004, 287: 0.004, 289: 0.0003, 290: 0.0029, 291: 0.0091, 293: 0.0002, 295: 0.0079, 299: 0.0015, 301: 0.0003, 302: 0.0023, 317: 0.0072, 318: 0.0004, 320: 0.0004, 322: 0.0009, 323: 0.0093, 327: 0.0069, 328: 0.0019, 330: 0.0058, 341: 0.0005, 342: 0.0019, 344: 0.0004, 345: 0.0153, 346: 0.0007, 347: 0.0011, 349: 0.0004, 351: 0.0411, 352: 0.0025, 356: 0.0104, 357: 0.0076, 358: 0.0055, 369: 0.0004, 373: 0.0063, 374: 0.0015, 377: 0.0164, 378: 0.0087, 379: 0.0092, 380: 0.0009, 384: 0.0061, 385: 0.0291, 387: 0.0006, 396: 0.0004, 399: 0.0072, 400: 0.0137, 402: 0.0009, 405: 0.0289, 406: 0.0596, 407: 0.0182, 409: 0.0023, 411: 0.0004, 412: 0.006, 415: 0.0005, 426: 0.0119, 428: 0.000